# 기상예보 데이터 수집

## 1. 동네예보 데이터 수집

라이브러리 임포트

In [30]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

동네 코드 정의

In [31]:
data_code = "1171056100"
data_name = "서울특별시 송파구 방이1동"
print(data_name)

서울특별시 송파구 방이1동


데이터 수집 소스

In [ ]:
japi = "http://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=" + data_code
print("데이터 소스 :", japi)

서울특별시 송파구 방이1동  동네 예보
데이터 소스 : http://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=1171056100


데이터 수집

In [ ]:
for data in weather.findAll('data'):
    hour = data.hour.string
    temp = data.temp.string
    reh = data.reh.string
    print("시간: %02s, 온도: %04s, 습도: %02s" % (hour, temp, reh ))

시간: 18, 온도: 20.0, 습도: 45
시간: 21, 온도: 17.0, 습도: 60
시간: 24, 온도: 15.0, 습도: 70
시간:  3, 온도: 14.0, 습도: 75
시간:  6, 온도: 14.0, 습도: 80
시간:  9, 온도: 16.0, 습도: 60
시간: 12, 온도: 21.0, 습도: 40
시간: 15, 온도: 23.0, 습도: 35
시간: 18, 온도: 21.0, 습도: 45
시간: 21, 온도: 18.0, 습도: 55
시간: 24, 온도: 16.0, 습도: 65
시간:  3, 온도: 14.0, 습도: 75
시간:  6, 온도: 13.0, 습도: 80
시간:  9, 온도: 16.0, 습도: 60
시간: 12, 온도: 21.0, 습도: 40
시간: 15, 온도: 23.0, 습도: 35
시간: 18, 온도: 21.0, 습도: 45
시간: 21, 온도: 18.0, 습도: 60
시간: 24, 온도: 16.0, 습도: 70


# 2. 동네예보 데이터 수집(오픈 API 사용)

In [32]:
import requests
from datetime import date, datetime

In [33]:
sky_code = ['맑음', '구름조금', '구름많음', '흐림']
today = date.today()
today_str = today.strftime('%Y%m%d') 
print('오늘날짜: ', today_str)

오늘날짜:  20201009


- 동네예보 오픈 API 키 발급 : https://data.go.kr/data/15057682/openapi.do
- 신규 발급 API키 승인에 1~2시간 소요
- 발급받은 API키로 변경 입력

In [34]:
ServiceKey ='58T8D9mCbvCPR5BywDitFmM8LeQFpwX3jJEMyBOMIKRDAsA5Ck%2FtdKvkjySUwQhGnnCQyuiJiU9K6oImDKRuaA%3D%3D'

In [35]:
pageNo =1
numOfRows = 10
dataType ='JSON'
base_date = today_str # 발표일 : 20200609
base_time ='0800' # 발표시간(1일 8회)
nx = 37
ny = 127

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService'
url += '/getVilageFcst?ServiceKey={}&pageNo={}&numOfRows={}&dataType={}&base_date={}&base_time={}&nx={}&ny={}'.format(ServiceKey, pageNo, numOfRows, dataType, base_date,base_time, nx, ny)

resp = requests.get(url)

In [36]:
print(resp)

<Response [200]>


In [37]:
print(resp.text)

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE"},"body":{"dataType":"JSON","items":{"item":[{"baseDate":"20201009","baseTime":"0800","category":"POP","fcstDate":"20201009","fcstTime":"1200","fcstValue":"20","nx":37,"ny":127},{"baseDate":"20201009","baseTime":"0800","category":"PTY","fcstDate":"20201009","fcstTime":"1200","fcstValue":"0","nx":37,"ny":127},{"baseDate":"20201009","baseTime":"0800","category":"R06","fcstDate":"20201009","fcstTime":"1200","fcstValue":"0","nx":37,"ny":127},{"baseDate":"20201009","baseTime":"0800","category":"REH","fcstDate":"20201009","fcstTime":"1200","fcstValue":"65","nx":37,"ny":127},{"baseDate":"20201009","baseTime":"0800","category":"S06","fcstDate":"20201009","fcstTime":"1200","fcstValue":"0","nx":37,"ny":127},{"baseDate":"20201009","baseTime":"0800","category":"SKY","fcstDate":"20201009","fcstTime":"1200","fcstValue":"3","nx":37,"ny":127},{"baseDate":"20201009","baseTime":"0800","category":"T3H","fcstDate":"20201009","fcstTime":"

In [38]:
if resp.status_code == 200:
    resp_json = resp.json()
    response = resp_json['response']
    result_msg = response['header']['resultMsg']

    if result_msg == 'NORMAL_SERVICE':
        body_data = response['body']
        for w in body_data['items']['item']:
            category = w['category']
            if category == 'SKY':
                sky = int(w['fcstValue'])
            elif category == 'T3H':
                temperature = float(w['fcstValue'])
            elif category == 'REH':
                humidity = float(w['fcstValue'])                
        print("하늘 : {}, 온도 : {}, 습도 : {}".format(sky_code[sky-1], temperature, humidity))
    else:
        print(result_msg)

하늘 : 구름많음, 온도 : 17.0, 습도 : 65.0
